In [17]:
import pandas as pd
import json
import os
import mapply
from google.cloud import bigquery
import gcsfs
import ast
import time
from tqdm import tqdm
client = bigquery.Client()
from vertexai.preview.language_models import TextGenerationModel
from vertexai.preview.generative_models import GenerativeModel
from smart_open import smart_open
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from langchain.agents import initialize_agent, Tool, load_tools, AgentType
from langchain.tools import BaseTool
from tqdm import tqdm
mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)
tqdm.pandas(desc='progress bar')
pd.set_option('display.max_colwidth', None)  # Set maximum width to terminal width
pd.set_option('display.expand_frame_repr', True)  # Expand DataFrame representation


In [3]:
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

/tmp/ipykernel_6626/111781775.py:1: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [4]:
data_frame = pd.read_sql_query(f"SELECT * FROM data_quality.meta_Prime_Pantry where asin not in ('B010RS31WS','B010RS1QO8','B00QTUEUEM','B01EMZ4HES') and rand()<0.01 ;",engine)
data_frame.shape

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/client.py:572: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


(107, 14)

In [5]:
sample_df = data_frame.drop_duplicates(subset=["title"]).sample(n=50,random_state=42).reset_index(drop=True)
sample_df.shape

(50, 14)

In [6]:
from vertexai.preview import generative_models
def generate_text(prompt):
    model = generative_models.GenerativeModel("gemini-pro")

    # Generation config
    config = {"max_output_tokens": 2048, "temperature": 0.0, "top_p": 1, "top_k": 32}

    # Safety config
    safety_config = {
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
                    }
    
    # Generate content
    responses = model.generate_content(
        [prompt],
        generation_config=config,
        stream=True,
        safety_settings=safety_config,
    )

    text_responses = []
    for response in responses:
        text_responses.append(response.text)
    return "".join(text_responses)


In [8]:
sample_df['attribute_extraction'] = sample_df['title'].progress_apply(lambda x: generate_text('''
You are an expert in extracting product quantity details from titles. Given a product title, identify relevant details and output them in the following JSON format with appropriate field labels:
Example product titles:
Title: V8 +Energy, Juice Drink with Green Tea, Peach Mango, 8oz. Can (4 packs of 6, Total of 24) 
Response: {'unit_of_measure': '8oz', 'count_per_pack': 6, 'multi_quantity': 4,'total_quantity': 24}

Title: V8, Strawberry Banana, 8 Ounce, 6 Count (Pack of 4) 
Response: {'unit_of_measure': '8 Ounce', 'count_per_pack': 4, 'multi_quantity': 6, 'total_quantity': 24}

Title: V8 100% Vegetable Juice, Spicy Hot, 5.5 oz. Can, 6 Count 
Response: {'unit_of_measure': '5.5 oz', 'count_per_pack': 1, 'multi_quantity': 6, 'total_quantity': 6}

Title: Orbit Peppermint Sugarfree Gum, 3 Pack
Response: {'unit_of_measure': null, 'count_per_pack': 1, 'multi_quantity': 3, 'total_quantity': 3}

Title : EO Botanical Sulfate Free Conditioner with Keratin, Coconut and Hibiscus, 8.4 oz
Response: {'unit_of_measure': '8.4 oz', 'count_per_pack': 1, 'multi_quantity': 1, 'total_quantity': 1}

Guidelines:
If count per pack is not given in the provided value and multi quantity is given, set count per pack to 1; if multi quantity is not given and count per pack is given, set multi quantity to 1.
Calculate total quantity if not explicitly given using (count per pack * multi quantity)

Now it's your turn

Title: ''' + x + '''
Response:'''))

progress bar: 100%|██████████| 50/50 [00:47<00:00,  1.05it/s]


In [18]:
sample_df['attribute_extraction']

0                   {"unit_of_measure": null, "count_per_pack": 1, "multi_quantity": 4, "total_quantity": 4}
1          {"unit_of_measure": "2&quot;", "count_per_pack": 742, "multi_quantity": 1, "total_quantity": 742}
2                 {"unit_of_measure": "8 Oz", "count_per_pack": 1, "multi_quantity": 1, "total_quantity": 1}
3          {"unit_of_measure": null, "count_per_pack": null, "multi_quantity": null, "total_quantity": null}
4             {"unit_of_measure": "1 Gallon", "count_per_pack": 1, "multi_quantity": 1, "total_quantity": 1}
5           {"unit_of_measure": "30.6 Ounce", "count_per_pack": 1, "multi_quantity": 1, "total_quantity": 1}
6                 {"unit_of_measure": null, "count_per_pack": 72, "multi_quantity": 1, "total_quantity": 72}
7            {"unit_of_measure": "1 Ounce", "count_per_pack": 1, "multi_quantity": 12, "total_quantity": 12}
8                   {"unit_of_measure": null, "count_per_pack": 1, "multi_quantity": 1, "total_quantity": 1}
9            {"unit

In [10]:
sample_df['attribute_extraction'] = sample_df['attribute_extraction'].apply(lambda json_str: json_str.replace("'", "\""))

In [11]:
# Assuming you have a DataFrame named 'df' with the column 'data_column'
sample_df['unit_of_measure'] = sample_df['attribute_extraction'].apply(lambda x: json.loads(x)['unit_of_measure'])
sample_df['count_per_pack'] = sample_df['attribute_extraction'].apply(lambda x: json.loads(x)['count_per_pack'])
sample_df['multi_quantity'] = sample_df['attribute_extraction'].apply(lambda x: json.loads(x)['multi_quantity'])
sample_df['total_quantity'] =sample_df['attribute_extraction'].apply(lambda x: json.loads(x)['total_quantity'])

# Now you can access these values as separate columns in your DataFrame


In [14]:
sample_df['verification'] = sample_df.progress_apply(lambda x: generate_text(f'''
Verify total quantity accuracy for the following product details:
count_per_pack: {x['count_per_pack']}
multi_quantity: {x['multi_quantity']}
total_quantity: {x['total_quantity']}
Apply this formula:
total_quantity = count_per_pack * multi_quantity
Respond with:
"Correct" if total_quantity is accurate.
The correctly calculated value if it's incorrect.
Response:'''),axis=1)


progress bar: 100%|██████████| 50/50 [00:24<00:00,  2.05it/s]


In [19]:
sample_df[sample_df['verification']!='Correct'][['attribute_extraction','verification']]

,attribute_extraction,verification
3,"{""unit_of_measure"": null, ""count_per_pack"": null, ""multi_quantity"": null, ""total_quantity"": null}","""Incorrect""\ntotal_quantity = nan"
18,"{""unit_of_measure"": null, ""count_per_pack"": null, ""multi_quantity"": null, ""total_quantity"": null}","""Incorrect""\ntotal_quantity = nan"
27,"{""unit_of_measure"": null, ""count_per_pack"": null, ""multi_quantity"": null, ""total_quantity"": null}","""Incorrect""\ntotal_quantity = nan"
28,"{""unit_of_measure"": ""38-Ounce"", ""count_per_pack"": 1, ""multi_quantity"": 1, ""total_quantity"": 38}",Incorrect\n38.0
36,"{""unit_of_measure"": ""fl. oz."", ""count_per_pack"": 1, ""multi_quantity"": 1, ""total_quantity"": 33.8}",Incorrect\n33.8
